In [ ]:
#load modules
import os
from scripts import gdrive
import subprocess
import glob

In [ ]:
#define the constants
#NOW = datetime.now() should be loaded by the previous cell in UI.ipynb

In [ ]:
#function 
def merge(filename, alert_map, glad_dir):
    """ merge into a single tif files"""
    
    #create command
    command = [
        'gdal_merge.py',
        '-o', alert_map,
        '-v', '-co', '"COMPRESS=LZW"'
    ]
    
    #add the input files
    for file in glob.glob(glad_dir + filename):
        command.append(file)

    process = subprocess.run(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    ) 
    
    print(process.stdout)
    
def clump(alert_map, clump_map):
    """ clump the results"""
    process = subprocess.run(
        [
            'oft-clump', 
            '-i', alert_map, 
            '-o', clump_map
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print (process.stdout)
    
def calc(clump_map, alert_map, alert_stats):
    """Compute the statistics per each individual clump"""
    process = subprocess.run(
        [
            'oft-his', 
            '-um', clump_map, 
            '-i', alert_map, 
            '-o', alert_stats,
            '-maxval','3'
        
        ],
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        universal_newlines=True
    )
    
    print(process.stdout)  
    
def create_result_folder():
    """Create a folder to download the glad images"""
    glad_dir = os.path.join(os.path.expanduser('~'), 'glad_results')+'/'
    if not os.path.exists(glad_dir):
        os.makedirs(glad_dir)
    
    return glad_dir

def download_task_tif(country_code, year):
    drive_handler = gdrive.gdrive()
    files = drive_handler.get_files(construct_filename(country_code, year)) #maybe I shoud use a global variable
    drive_handler.download_files(files, glad_dir)
    
def delete_source_file(filename, glad_dir):
    #list the input files
    file_list = []
    for file in glob.glob(glad_dir + filename):
        file_list.append(file)
        
    for file in file_list:
        os.remove(file)

In [ ]:
#run 
glad_dir = create_result_folder()
download_task_tif(country_code, year)

#define the files variables
#year and country_code are defined by process cell
alert_map   = glad_dir + "glad_" +year + "_" + country_code + ".tif"
clump_map   = glad_dir + "tmp_clump_" + year + "_" + country_code + ".tif"
alert_stats = glad_dir + "stats_glad_" + year + "_" + country_code + ".txt"

#process data with otf
filename = construct_filename(country_code, year) + "*.tif"
merge(filename, alert_map, glad_dir)
clump(alert_map, clump_map)
calc(clump_map, alert_map, alert_stats)